In [1]:
import yfinance as yf
import pandas as pd
import polars as pl
import os
import re
import urllib.parse
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math 
import pyarrow
import numpy as np

Collect scores from hard memory


In [2]:
root = r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\data\snp_500_scores'

ticker_symbol_pattern = re.compile(r'^([A-Z]+(?:\.[A-Z]+)?)\.parquet$')

df_dict_scores = {}

for dir in os.listdir(root):
    
    match = ticker_symbol_pattern.search(dir)

    ticker = match.group(1)
    ticker = ticker.replace('.', '-')
    print(ticker)

    file_path = os.path.join(root, dir)

    df = pl.read_parquet(file_path)
    df_dict_scores[ticker] = df 


A
AEE
AKAM
ALGN
AME
AMT
AMZN
ANSS
ARE
AXON
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BX
BXP
C
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTLT
CTSH
CVS
CVX
CZR
D
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVN
DXCM
EBAY
ECL
ED
EFX
EG
EIX
EL
EMN
EMR
ENPH
EOG
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HD
HES
HIG
HII
HLT
HOLX
HPE
HPQ
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INTC
INTU
INVH
IP
IPG
IQV
IR
IRM
ISRG
IT
ITW
IVZ
J
JBHT
JBL
JCI
JKHY
JNJ
JNPR
JPM
K
KDP
KEY
KEYS
KHC
KIM
KMB
KMI
KMX
KO
KR
KVUE
L
LDOS
LEN
LH
LHX
LIN
LKQ
LLY
LMT
LNT
LOW
LRCX
LULU
LUV
LVS
LW
LYB
LYV
MAR
MCD
MCHP
MCK
MCO
MDLZ
MDT
MET
MGM
MHK
MKC
MKTX
MLM
MMC
MMM
MNST
MO
MOH
MOS
MPC
MRK
MRO
MS
MSCI
MSF

This is support func to find end prices and their dates. It takes as an arguments starting index, which is the nuber of row of hist dataframe and this hist dataframe itself. Hist dataframe contain daily OHLC data for particular ticker (company). 

It also cheks wether index of end price is in hist, meaning that if this particular report was released very resently (less then 8 trading days ago) it will append none values for end prices out of range of hist df instead of raising an error.

It returns lists of end prices and corresponding dates (none values for prices and their dates, if they are out of range of hist df).

In [3]:
def find_end_price(start_index, hist):
    end_price_list = []
    end_price_date_list = []
    
    for x in range(2, 8):
        idx = start_index + x
        if idx < len(hist):
            end_price_list.append(hist.iloc[idx]['Open'])
            end_price_date_list.append(hist.index[idx])
        else:
            end_price_list.append(None)
            end_price_date_list.append(None)
    return end_price_list, end_price_date_list

This is support func to calc the regular returns. It takes start price and end_price_list as an arguments. It checks whether value in end price list is none and if it is, it appends none in returns list, instead of raising an error. So we are shure that all end price lists are of the same length, but for those prices that are not found in historical data we have none values for returns.

In [4]:
def regular_returns(start_price, end_price_list):
    retuns = []
    for end_price in end_price_list:
        if end_price is None:
            retuns.append(None)
        else:
            ret = (end_price - start_price) / start_price * 100
            retuns.append(ret)
    return retuns

This is the support func that calc SnP500 returns. It takes_start date and end_price_date_list as and arguments and then calculate returns for this timeframes. This func also checks whether value of end_price_date_list is none (meaning that it is not found in historical data), and if it is, func will append none instead of raising an error.

Now we are shure that list of snp_returns will be of equal length for all report release dates and all time frames, but for those returns that cannot be calculates due to non existent end prices we will have nones 

In [5]:
def snp_500_return(start_date, end_price_date_list, snp_price):

    snp_returns = []

    for end_date in end_price_date_list:
        if end_date is None:
            snp_returns.append(None)

        else: 
            start_index = snp_price.index.get_loc(start_date)
            start_price = snp_price.iloc[start_index]['Open']
    
            end_index = snp_price.index.get_loc(end_date)
            end_price = snp_price.iloc[end_index]['Open']
    
            ret = (end_price - start_price) / start_price * 100
            
            snp_returns.append(ret)

    return snp_returns

This is suppurt function that retrive the end quarter price. It take as an argument hist - historical price dataframe, date_str - the date of release of this particular report, df - company dataframe, with cols named with dates of releases of corresponding reports.

It takes the release date for current report and checks whether it is the last date in time series or not. If it is (this mean that we are are realy close to the end of hist df) we consider the end of hist df as the end of quarter. if it is not, we take next date in company df (which is the date of releas of the next quarterly report) and see its open price, this is pricisely the end of the quarter.

In [6]:
def find_end_quarter(hist, date_str, start_date, df):

    col_names = df.columns
    current_date_index = col_names.index(date_str)

    if current_date_index < len(col_names) - 1:

        next_date = col_names[current_date_index + 1]

        next_date_time_stemp = pd.Timestamp(next_date, tz='America/New_York')

        while next_date_time_stemp not in hist.index:

            next_date_time_stemp += pd.Timedelta(days=1)
    
        end_quarter_index = hist.index.get_loc(next_date_time_stemp)
        end_quarter_price = hist.iloc[end_quarter_index]['Open']
        end_quarter_date = hist.index[end_quarter_index]

        start_quarter_index = hist.index.get_loc(start_date)

        quarter_length = end_quarter_index - start_quarter_index

    else:
        end_quarter_price = None
        end_quarter_date = None
        quarter_length = None
    

    return end_quarter_price, end_quarter_date, quarter_length

This is a func that takes as an input a key-value pair from df_dict_scores and calculate 2-days, 3-days, 4-days, 5-days, 6-days, 7-days and full quarter excess returns for it (S&P500 is the benchmark). It stores the result as a polars df with dates of starting of the period as col names and corresponding returns as col values (seven in each: starting from 2-days down to 7-days and full quarter) - 

In [8]:
def computing_returns(tic_sym, df):

    company_ticker = yf.Ticker(tic_sym)
    hist = company_ticker.history(period="max")

    snp500_ticker = yf.Ticker("^GSPC")
    snp_price = snp500_ticker.history(period="max", auto_adjust=True)
    
    returns = {}

    for date_str in df.columns:

        start_date = pd.Timestamp(date_str, tz='America/New_York')
        
        while start_date not in hist.index:

            start_date += pd.Timedelta(days=1)
        

        start_index = hist.index.get_loc(start_date)

        start_price = hist.iloc[start_index]['Open']

        end_price_list, end_price_date_list = find_end_price(start_index, hist)
        
        if None not in end_price_list:
            end_quarter_price, end_quarter_date, length_of_quarter = find_end_quarter(hist, date_str, start_date, df)
            
        else:
            end_quarter_price = None
            end_quarter_date = None

        
        end_price_list.append(end_quarter_price)
        end_price_date_list.append(end_quarter_date)
        
        reg_returns = regular_returns(start_price, end_price_list)
        snp_returns = snp_500_return(start_date, end_price_date_list, snp_price)
        
        excess_returns = [a - b if a is not None and b is not None else None for a, b in zip(reg_returns, snp_returns)]
        
        #normalization: divide returns for each time frame by number of trading days (2-days returns/2, 3-days returns/3, ful quarter returns/length_of_quarter)
        timeframe_length = [2, 3, 4, 5, 6, 7, length_of_quarter]

        normalized_excess_returns = [x / y if x is not None and y is not None else None for x, y in zip(excess_returns, timeframe_length)]

        returns[date_str] = normalized_excess_returns

    returns = pl.DataFrame(returns)
    
    return returns


In [10]:
#example ticker symbol and correponding df
tic_sym = "GPC"
df = df_dict_scores[tic_sym]
df

2019-02-25,2019-04-19,2019-07-19,2019-10-18,2020-02-21,2020-05-06,2020-07-30,2020-10-22,2021-02-19,2021-04-22,2021-07-22,2021-10-21,2022-02-17,2022-04-21,2022-07-27,2022-10-20,2023-02-23,2023-04-20,2023-07-20,2023-10-19
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.101224,-0.04918,-0.031963,-0.030303,-0.227493,-0.223796,-0.295405,-0.29087,-0.260903,-0.077519,-0.097179,-0.10241,-0.229299,-0.115385,-0.010417,0.02439,-0.16792,0.0,0.043478,0.010309
-91.0,-9.0,-7.0,-7.0,-235.0,-79.0,-135.0,-137.0,-341.0,-20.0,-31.0,-34.0,-288.0,-24.0,-2.0,5.0,-201.0,0.0,8.0,2.0
0.57363,0.577068,0.562252,0.560505,0.559737,0.500294,0.469084,0.490036,0.533217,0.581062,0.55205,0.545511,0.546587,0.547689,0.552709,0.55408,0.497703,0.577011,0.543784,0.513007
2941.0,614.0,709.0,755.0,2975.0,851.0,1009.0,1082.0,3347.0,810.0,875.0,881.0,3291.0,557.0,561.0,584.0,2600.0,502.0,503.0,493.0
30577.0,6884.0,8300.0,8881.0,30650.0,10265.0,12664.0,13004.0,35747.0,8316.0,9475.0,9674.0,34138.0,7097.0,7248.0,7562.0,31666.0,6274.0,6781.0,7112.0


In [11]:
computing_returns(tic_sym, df)

2019-02-25,2019-04-19,2019-07-19,2019-10-18,2020-02-21,2020-05-06,2020-07-30,2020-10-22,2021-02-19,2021-04-22,2021-07-22,2021-10-21,2022-02-17,2022-04-21,2022-07-27,2022-10-20,2023-02-23,2023-04-20,2023-07-20,2023-10-19
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.521221,-0.46944,-0.580699,1.254158,0.233216,-0.425536,-0.691618,-1.410751,2.404138,0.676916,-1.878107,1.300041,-0.226948,0.658194,-0.410692,-0.423191,1.931913,0.149507,-1.043665,-3.653592
-0.501408,-0.617147,-0.052012,1.480307,0.195308,-0.044748,-0.537198,-1.312531,1.412172,0.331834,-1.475566,0.834692,-0.778055,0.101624,0.017442,0.690328,1.002422,0.219518,-0.744778,-2.390267
-0.371891,-0.525883,-0.128087,1.237687,-0.325789,-0.135327,-0.136816,-1.376712,1.149428,0.457683,-0.827269,0.712174,-1.018908,0.818473,0.234385,0.718173,0.353772,-0.030854,-0.540367,-1.878629
-0.405512,-0.596882,-0.012629,0.688587,-0.256303,-0.221513,-0.182952,-1.149393,1.286282,0.553159,-0.967433,0.35454,-0.73423,0.469663,-0.028841,0.53963,-0.254195,-0.022062,-0.515215,-1.240333
-0.315083,-0.634719,-0.028279,0.492831,-0.005001,-0.763556,-0.292982,-1.015858,1.090905,0.541325,-0.714551,0.126003,-0.736158,0.112584,-0.294908,0.877188,-0.15056,0.169173,-0.429714,-0.909529
-0.274639,-0.599863,0.099691,0.458825,-0.411147,-0.656573,-0.174843,-0.597258,0.953794,0.516846,-0.61201,0.021006,-0.609347,0.306438,-0.294929,0.683283,-0.314905,0.112915,-0.421916,-0.874399
-0.214851,-0.164345,0.041571,-0.13297,-0.180509,0.161621,0.095863,-0.147301,0.31101,0.085233,-0.082369,0.036754,0.154187,0.267591,0.302136,-0.00226,-0.191209,-0.207123,-0.089662,null


This is the general loop, that takes each key-value pair from df_dict_scores, applies computing_returns() to it, then verticaly stackes the result with orig df and stores it in the new dict

In [12]:
dict_for_anal = {}
row_names = pl.Series("row_names", ["lm_polarity", "lm_raw", "hiv4_polarity", "hiv4_raw", "doc_length", "2_day_reterns","3_day_reterns", "4_day_reterns", "5_day_reterns", "6_day_reterns", "7_day_reterns", "full_quarter_returns"])

for tic_sym, df in df_dict_scores.items():
    print(tic_sym)

    returns_df = computing_returns(tic_sym, df)
    
    stacked_df = df.vstack(returns_df)
    stacked_df_with_indx = stacked_df.hstack([row_names])
    
    dict_for_anal[tic_sym] = stacked_df_with_indx

A
AEE
AKAM
ALGN
AME
AMT
AMZN
ANSS
ARE
AXON
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BX
BXP
C
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTLT
CTSH
CVS
CVX
CZR
D
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVN
DXCM
EBAY
ECL
ED
EFX
EG
EIX
EL
EMN
EMR
ENPH
EOG
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HD
HES
HIG
HII
HLT
HOLX
HPE
HPQ
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INTC
INTU
INVH
IP
IPG
IQV
IR
IRM
ISRG
IT
ITW
IVZ
J
JBHT
JBL
JCI
JKHY
JNJ
JNPR
JPM
K
KDP
KEY
KEYS
KHC
KIM
KMB
KMI
KMX
KO
KR
KVUE
L
LDOS
LEN
LH
LHX
LIN
LKQ
LLY
LMT
LNT
LOW
LRCX
LULU
LUV
LVS
LW
LYB
LYV
MAR
MCD
MCHP
MCK
MCO
MDLZ
MDT
MET
MGM
MHK
MKC
MKTX
MLM
MMC
MMM
MNST
MO
MOH
MOS
MPC
MRK
MRO
MS
MSCI
MSF

In [13]:
dict_for_anal["COST"]

2019-03-13,2019-06-06,2019-10-11,2019-12-23,2020-03-12,2020-06-04,2020-10-07,2020-12-16,2021-03-10,2021-06-03,2021-10-06,2021-12-22,2022-03-10,2022-06-02,2022-10-05,2022-12-29,2023-03-09,2023-06-01,2023-10-11,2023-12-20,row_names
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.206751,-0.184314,-0.313433,-0.34413,-0.304348,-0.378747,-0.360294,-0.362712,-0.396721,-0.358255,-0.380385,-0.35082,-0.309211,-0.339394,-0.358279,-0.372671,-0.378049,-0.368421,-0.365762,-0.340351,"""lm_polarity"""
-49.0,-47.0,-315.0,-85.0,-77.0,-139.0,-392.0,-107.0,-121.0,-115.0,-415.0,-107.0,-94.0,-112.0,-383.0,-120.0,-124.0,-126.0,-391.0,-97.0,"""lm_raw"""
0.486711,0.447115,0.429211,0.478873,0.454098,0.401434,0.408528,0.398825,0.381762,0.350338,0.411323,0.408146,0.365539,0.337433,0.407461,0.382573,0.347928,0.332814,0.409683,0.430544,"""hiv4_polarity"""
586.0,558.0,1725.0,544.0,554.0,672.0,1782.0,475.0,494.0,467.0,1751.0,491.0,454.0,439.0,1682.0,461.0,445.0,427.0,1667.0,499.0,"""hiv4_raw"""
8407.0,8594.0,23602.0,7720.0,8170.0,11161.0,25803.0,8219.0,8994.0,9249.0,25592.0,8117.0,8374.0,8776.0,24367.0,8030.0,8634.0,8683.0,24150.0,7741.0,"""doc_length"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.016938,0.48649,-0.252665,-0.168993,3.412315,-1.169852,0.140993,-0.487917,0.139892,-0.210352,0.068194,0.220163,0.814457,0.274489,0.201693,-0.285079,0.456089,-0.357984,0.436089,0.176786,"""4_day_reterns"""
0.011902,0.374856,0.035756,0.07349,3.941929,-0.325292,0.470364,-0.454021,0.006786,-0.143979,0.029712,0.266878,0.338515,0.248979,0.575442,0.232835,0.494229,-0.285085,0.363466,0.182815,"""5_day_reterns"""
0.054191,0.462862,0.087738,-0.144237,2.893316,-0.221541,0.398747,-0.427808,0.013736,-0.019404,-0.13982,0.103534,0.34439,0.640764,0.233863,0.537127,0.220959,-0.19557,0.433766,0.04174,"""6_day_reterns"""


This func takes as input key-value pair (company name and dataframe with sentiment scores and returns), regress each retern (2_days, 3_days, etc) on polarity score and raw sentiment score. Then it collects the slopes (beta-coefficient) of returns regressed on each score in the separate polars series (in the loop they will be staked in dataframe)

In [14]:
def add_quarter_dummy(company_df_pd):
    
    dates = pd.to_datetime(company_df_pd.columns)
   
    date_series = pd.Series(dates.quarter, index=dates)
   
    dummies = pd.get_dummies(date_series, prefix="Q")
    dummies.index = dummies.index.strftime('%Y-%m-%d')
    
    all_quarters = ["Q_1", "Q_2", "Q_3", "Q_4"]
    dummies = dummies.reindex(columns=all_quarters, fill_value=False)
   
    dummies = dummies.drop(columns=["Q_4"])
    
    dummies = dummies.transpose()
    df_wit_dum = pd.concat([company_df_pd, dummies])
    
    df_wit_dum = df_wit_dum.transpose()
   
    for col in df_wit_dum.columns:
        df_wit_dum[col] = pd.to_numeric(df_wit_dum[col])

    boolean_cols = ['Q_1', 'Q_2', 'Q_3']
    df_wit_dum[boolean_cols] = df_wit_dum[boolean_cols].astype(int)

    return df_wit_dum

    

In [24]:
def fit_regression(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    
    slope = model.params.iloc[1]
    print(model.summary())
    
    return slope

In [27]:
def regression_slopes(df_wit_dum, X_names, company_name="total_df"):

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; -3, -2, -1 columns represent dummy variables for quarters
    X = [df_wit_dum.iloc[:, [x, 4, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(5, 12)]

    slopes_dict = {}

    for i, x in enumerate(X):

        name_of_score = X_names[i][0].split()[0]
        slopes_name = f'{name_of_score}_slopes'
        print('\n', slopes_name)

        slopes = []
        for y in Y:
            slope = fit_regression(y, x)
            slopes.append(slope)

        slopes_dict[slopes_name] = pl.Series(company_name, slopes)
    
    return slopes_dict

This is the general loop that takes dict_for_anal whith scores and returns and for each company in the dict it calculates dict with four dfs: first one contain regression slopes for lm_polarity scores, second one for lm_raw scores and so on for each regressor. Serieses stacked in the separete dataframes (for raw and for polarity scores)

In [17]:
def compute_all_slopes(dict_for_anal):

    slopes_df_dict = {
        "lm_polarity_slopes_df" : pl.DataFrame(),
        "lm_raw_slopes_df" : pl.DataFrame(),
        "hiv4_polarity_slopes_df" : pl.DataFrame(),
        "hiv4_raw_slopes_df" : pl.DataFrame()
    }
    
    for company_name, company_df in dict_for_anal.items(): 

        X_names = [company_df.select(pl.col("row_names")).row(x) for x in range(0, 4)]

        company_df_woi = company_df.select(pl.exclude("row_names"))

        company_df_woi = company_df_woi.to_pandas()
        print(company_name)

        df_wit_dum = add_quarter_dummy(company_df_woi)
        if df_wit_dum.shape[0] > 6:

            slopes_dict = regression_slopes(df_wit_dum, X_names, company_name)
    
            for x in X_names:
                
                name_of_score = x[0].split()[0]
                slopes_name = f'{name_of_score}_slopes'
                
                df_name = f"{name_of_score}_slopes_df"
                
                slopes_df_dict[df_name] = slopes_df_dict[df_name].hstack([slopes_dict[slopes_name]])

    return slopes_df_dict
        

In [18]:
slopes_df_dict = compute_all_slopes(dict_for_anal)

A
AEE
AKAM
ALGN
AME
AMT
AMZN
ANSS
ARE
AXON
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BX
BXP
C
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTLT
CTSH
CVS
CVX
CZR
D
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVN
DXCM
EBAY
ECL
ED
EFX
EG
EIX
EL
EMN
EMR
ENPH
EOG
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HD
HES
HIG
HII
HLT
HOLX
HPE
HPQ
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INTC
INTU
INVH
IP
IPG
IQV
IR
IRM
ISRG
IT
ITW
IVZ
J
JBHT
JBL
JCI
JKHY
JNJ
JNPR
JPM
K
KDP
KEY
KEYS
KHC
KIM
KMB
KMI
KMX
KO
KR
KVUE
L
LDOS
LEN
LH
LHX
LIN
LKQ
LLY
LMT
LNT
LOW
LRCX
LULU
LUV
LVS
LW
LYB
LYV
MAR
MCD
MCHP
MCK
MCO
MDLZ
MDT
MET
MGM
MHK
MKC
MKTX
MLM
MMC
MMM
MNST
MO
MOH
MOS
MPC
MRK
MRO
MS
MSCI
MSF

As an output from previous functions we have dataframes with regression slopes of different returns on raw, polarity scores and Loughran-McDonald and Harvard dictionaries. Each col of these dfs contain regression slopes for particular company. First row contain slope for 2_days return, second row contain slope for 3_days return, etc.  I have decided to check what is the proportion of positive regression slopes for each return timeframe (2_days, 3_days, etc). For that purpose i will transpose each of these dfs, apply condition > 0, then verticaly sum boolean values and obtain the prorortion of positive slopes for each time frame. 

In [19]:
def calc_efficiency_metrics(slopes_df):

    df_tranposed = slopes_df.transpose()

    res = df_tranposed.select(pl.all() > 0).sum()
    total_num_of_slopes = slopes_df.shape[1]

    positive_slopes = [res.select(pl.col(res.columns[x]).gather(0)).item() for x in range(slopes_df.shape[0])]

    for counter, x in enumerate(positive_slopes):
        print(f'Prorortion of positive {counter + 2}_day returns: {x/total_num_of_slopes}')

This loop apply calc_efficiency_metrics function to each item of slopes_df_dict where each item is a dataframe which regression slopes for each regressor.

In [20]:
for key, slopes_df in slopes_df_dict.items():
    print("\n")
    print(f'Metrics for: {key}')
    calc_efficiency_metrics(slopes_df)



Metrics for: lm_polarity_slopes_df
Prorortion of positive 2_day returns: 0.4859335038363171
Prorortion of positive 3_day returns: 0.5012787723785166
Prorortion of positive 4_day returns: 0.5038363171355499
Prorortion of positive 5_day returns: 0.5294117647058824
Prorortion of positive 6_day returns: 0.4859335038363171
Prorortion of positive 7_day returns: 0.4782608695652174
Prorortion of positive 8_day returns: 0.4629156010230179


Metrics for: lm_raw_slopes_df
Prorortion of positive 2_day returns: 0.4782608695652174
Prorortion of positive 3_day returns: 0.47570332480818417
Prorortion of positive 4_day returns: 0.4936061381074169
Prorortion of positive 5_day returns: 0.4629156010230179
Prorortion of positive 6_day returns: 0.46547314578005117
Prorortion of positive 7_day returns: 0.4782608695652174
Prorortion of positive 8_day returns: 0.46035805626598464


Metrics for: hiv4_polarity_slopes_df
Prorortion of positive 2_day returns: 0.4833759590792839
Prorortion of positive 3_day retur

Positive slopes means that sentiment scores and returns mooves in the same direction (hense, they are working as predictors of stock price movement). As the result of this preliminary analisis we can say that sentiment scores of quarterly reports are not very good predictors of stock movments. Positive proportion of this slopes are close to 0.5, which means that on average in our sample there are 50/50 chance that scores and returns are mooving in the same direction.

As we can see that, despite visual conclusion that weighted scores are more "smooth", they barely outperform raw scores, but at least they are not worse, meaning that at each time frame (except 5_days) the proportion of positive regression slopes of weighted scores => proportion of positive regression slopes of raw scores.

This func stackes all data in one df, and calculate the single slope for each time frame

In [28]:
def compute_signl_slope(dict_for_anal):

    df_list = []
    
    for company_name in dict_for_anal:


        X_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(0, 4)] 
    
        company_df_woi = dict_for_anal[company_name].select(pl.exclude("row_names"))
        company_df_pd = company_df_woi.to_pandas(use_pyarrow_extension_array=True)

        df_wit_dum = add_quarter_dummy(company_df_pd)

        df_list.append(df_wit_dum)
        

    total_df = pd.concat(df_list, ignore_index=True)
    
    slopes_dict = regression_slopes(total_df, X_names)

    return slopes_dict

In [29]:
slopes_dict = compute_signl_slope(dict_for_anal)
print(slopes_dict)


 lm_polarity_slopes
                            OLS Regression Results                            
Dep. Variable:                      5   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.073
Date:                Thu, 16 May 2024   Prob (F-statistic):            0.00895
Time:                        13:27:58   Log-Likelihood:                -15802.
No. Observations:                7807   AIC:                         3.162e+04
Df Residuals:                    7801   BIC:                         3.166e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1127      0.06

jupyter nbconvert --to html Parsim-sec\src\Analysis\excess_ret_500.ipynb